In [23]:
!cd

C:\Users\MR ARCHITECT\MyFloorPlan\MyGenNew\MyGenFloorPlan\utils


In [24]:
root = '/Users/MR ARCHITECT/MyFloorPlan/MyGenNew/MyGenFloorPlan/'

In [25]:
!pip install Augmentor
!pip install numpy 
!pip install Pillow 

In [26]:
import Augmentor
import os
import shutil
from PIL import Image
import numpy as np

Documentation on Augmentor can be found here: https://github.com/mdbloice/Augmentor

In [27]:
pathA = root + '/dataset/Original/A'
pathB = root + '/dataset/Original/B' 

In [28]:
p = Augmentor.Pipeline(pathA)
# Point to a directory containing ground truth data.
# Images with the same file names will be added as ground truth data
# and augmented in parallel to the original data.
p.ground_truth(pathB)

# Add operations to the pipeline:
p.rotate90(probability=0.25)
p.rotate270(probability=0.25)
p.flip_left_right(probability=0.25)
p.flip_top_bottom(probability=0.25)


Initialised with 5 image(s) found.
Output directory set to /Users/MR ARCHITECT/MyFloorPlan/MyGenNew/MyGenFloorPlan//dataset/Original/A\output.

5 ground truth image(s) found.


In [29]:
IMAGE_COUNT = 300

p.sample(IMAGE_COUNT)

Processing <PIL.Image.Image image mode=RGBA size=256x256 at 0x1DF92973090>: 100%|██████████| 300/300 [00:00<00:00, 522.98 Samples/s]                


In [30]:
# Wait a few minutes before continuing to make sure the output images get uploaded to Google Drive, since we'll be 
# copying and combining them in the next step

In [35]:
newPathA = root + '/dataset/Augmented/A_processed/'
newPathB = root + '/dataset/Augmented/B_processed/'
combinedPath = root + '/dataset/Augmented/Combined_2024_03_18_15_41/'

In [36]:
# Iterate over the files generated by the Augmentor pipeline and copy them to a new directory
# The empty destination directory needs to exist before running this cell

# This is the default location that Augmentor will have saved the sampled images
os.chdir(pathA + '/output')

A_files = []
B_files = []

for filename in os.listdir('.'):
  if 'A_original' in filename:
      # guid will include the .png suffix
      guid = filename[-16:]
      A_files.append(guid)
      # shutil.copyfile(src, dst, *, follow_symlinks=True)
      dst = newPathA + guid
      shutil.copyfile(filename, dst)
  if '_groundtruth_' in filename:
      # guid will include the .png suffix
      guid = filename[-16:]
      B_files.append(guid)
      # shutil.copyfile(src, dst, *, follow_symlinks=True)
      dst = newPathB + guid
      shutil.copyfile(filename, dst)

print("A Filenames Found: ", A_files)
print("B Filenames Found: ", B_files)

A Filenames Found:  ['be40b7c2bc25.png', '7f490d1f0c23.png', '5f4db78db599.png', '3aa25ccddecf.png', '3f187f1e447a.png', '4e293b45e2a9.png', '5d2ef2e3905d.png', '3acbfb56252f.png', '6ec336d4c8c3.png', '1d9e6b1ae650.png', 'a880c25b6cfe.png', 'ac019f61cbe8.png', '9f7052e42b76.png', 'c8d5a23b135f.png', 'd9d95c858b2e.png', '54c78cb3a1dc.png', 'a5d82dbf776a.png', '70e7646f3582.png', 'ff7b503c94c7.png', 'c2968af9d4e2.png', 'acc89250c909.png', 'd72c0838aa75.png', 'abcaba6732e8.png', 'cfa65be46953.png', '08f9e056bb1a.png', 'f674d8e3e9c4.png', 'df4bb84155e6.png', '8c987c4b2fa7.png', '90d7df0995b4.png', 'ea48b81ac9a8.png', 'ed57ac8087a6.png', '149286c0fa6c.png', 'ce665b3904d0.png', '6aa82c9c836b.png', '6756f2bb6200.png', 'e77c973f10a1.png', 'a13c3dab8a1d.png', '89a171f40cc0.png', 'ec47f32abc87.png', 'f257a98d2a7b.png', '0f0b1259c5af.png', 'c99764d7a0f7.png', '47f458e603f5.png', 'becc916e7d22.png', 'f3ced65f777d.png', '80fb9cb70f8e.png', '5100e135f25c.png', 'f8ea1eec77a8.png', 'ec91cd78d61c.png',

In [37]:
# Combine Augmented Images into the pair format that Pix2Pix expects
os.chdir(newPathA)
for filename in os.listdir('.'):
  im1 = Image.open(filename)
  im2 = Image.open(newPathB + filename)

  # Combine images horizontally using PIL
  dst = Image.new('RGB', (512, 256))
  dst.paste(im1, (0, 0))
  dst.paste(im2, (im1.width, 0))
  dst.save(combinedPath + filename)

In [38]:
# Train/Test Split
# Takes the files in '../Augmented/Combined/' and randomly splits them into './Combined/Train' and './Combined/Test' directories
os.chdir(combinedPath)
test_dir = combinedPath + 'Test/'
train_dir = combinedPath + 'Train/'

files = os.listdir(combinedPath)
ratio = 0.15

for f in files:
    if '.png' in f:
      if np.random.rand(1) < ratio:
          shutil.copyfile(combinedPath + f, test_dir + f)
      else:
          shutil.copyfile(combinedPath + f, train_dir + f)

In [39]:
# Check that files exist in new location:

# os.chdir(pathA+ '/output')
os.chdir(train_dir)
print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

258
